In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher
from ipynb.fs.full.helper import get_text_from_csv,clean_text,check_null,find_words,find_topic,get_word_lable,count_topic,count_word


# GOOGLE + FACEBOOK

In [28]:
def get_text_from_csv(fn): # get ad's text from csv file, return a dataframe with text, ad_id,ad_url, and report_url
    df = pd.read_csv(fn)
    df = df[df['ad_type'] == 'Text'].reset_index(drop = True) # only get text ads
    df_url = df[['ad_id','ad_url']].reset_index(drop = True) # get the urls
    urls = df_url['ad_url'].to_list() # create a list so we could get the report_urls
    report_urls = []
    for url in urls:
        entity_id = url.split('/')[-3]
        creative_id = url.split('/')[-1]
        report_url = 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id={}&creative_id={}&hl=en'.format(entity_id,creative_id)
        report_urls.append(report_url)
    ad_text = [] # get texts from reports
    for report_url in report_urls:
        response = requests.get(report_url)
        text = response.text.split('"]')[0].split('[')[-1]
        ad_text.append(text)  
    df_text = pd.DataFrame(ad_text).rename(columns = {0:'text'})
    df_new = df[df['ad_type']=='Text'][['advertiser_name']].reset_index(drop = True) 
    df = pd.concat([df_text,df_new],axis=1)
    return df  

In [29]:
# read google ads
google_df = get_text_from_csv('GoogleAds/CA.csv') 
google_df.head()

,text,advertiser_name
0,"""Ad Paid for by Dave Myers for San Diego County Sheriff 2022 | FPPC ID # 1436219"",""Don't Settle for the Status Quo! Elect Dave Myers for Sheriff come June 7th, 2022!",Dave Myers for Sheriff 2022
1,"""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."",""Stop The Deceptive Corp Prop: Would Turn Every Screen Into A Gambling Device. Learn More.","Coalition for Safe, Responsible Gaming, Sponsored by California Indian Tribes"
2,"""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."",""Stop The Deceptive Corp Prop: Would Turn Every Screen Into A Gambling Device. Learn More.","Coalition for Safe, Responsible Gaming, Sponsored by California Indian Tribes"
3,"""Stand with Captain Sam Brown. Let's restore accountability and leadership to Washington."",""Captain Sam Brown fought for our Country. Now, he is ready to serve the people of Nevada.",SAM BROWN FOR NEVADA
4,"""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."",""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA.","Coalition for Safe, Responsible Gaming, Sponsored by California Indian Tribes"


In [36]:
# read facebook ads
facebook_df = pd.read_csv('FacebookAds/california.csv')

In [37]:
# only keep text column
facebook_df.drop(columns = ['Unnamed: 0'],inplace = True)
facebook_df.rename(columns = {'byline':'advertiser_name'},inplace = True)
pd.set_option('display.max_colwidth', None)
facebook_df

,text,advertiser_name
0,It is awesome to have the support of the California Nurses Association! What they have been through in this pandemic is unreal… If hospitals are going to continue not paying property taxes they must treat our nurses better!,Braden Murphy for State Board of Equalization 2022
1,"When the other candidate rides the coat tails of other establishment politicians, I ride in on my experience, dedication and support from the people…. Transparency and Credibility #ad5 #california",Jason Paletta for CA Secretary of State
2,"Seydi has worked with San Fernando Valley communities for 20 years. She understands the issues we are facing and she's prepared to advocate for us in Sacramento. \n\nVote for Seydi because she pledges toward fixing:\n\n📌 Homelessness\n📌 Housing\n📌 Education\n📌 Small Business\n📌 Access to healthcare\n\nMost importantly, she pledges to listen to her constituents. Vote for Seydi Alejandra Morales for State Senate District 20 starting May 9th!",Seydi Alejandra Morales for Senate 2022
3,Learn more about our efforts to find solutions that will protect our planet while building a more equitable future for CA families.,Environmental Solutions Coalition
4,Special interests like 𝗥𝗲𝗰𝗼𝗹𝗼𝗴𝘆 are exploiting environmental problems for their own profits. \n\nThe San Francisco-based garbage and recycling giant was fined $36 million in a public corruption scandal after it admitted to bribery allegations related to overcharging customers $100 million. Don’t let special interests make up for lost profits by raising taxes on California taxpayers.,Environmental Solutions Coalition
...,...,...
10043,Help Protect Atkins Place!,Save The Redwoods League
10044,Help Protect Atkins Place!,Save The Redwoods League
10045,Help Protect Atkins Place!,Save The Redwoods League
10046,Help Protect Atkins Place!,Save The Redwoods League


In [38]:
# cncat google_df and facebook_df by 'text'
df_text = pd.concat([google_df,facebook_df])
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10198 entries, 0 to 10047
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             10196 non-null  object
 1   advertiser_name  10174 non-null  object
dtypes: object(2)
memory usage: 239.0+ KB


In [39]:
df_text.head()

,text,advertiser_name
0,"""Ad Paid for by Dave Myers for San Diego County Sheriff 2022 | FPPC ID # 1436219"",""Don't Settle for the Status Quo! Elect Dave Myers for Sheriff come June 7th, 2022!",Dave Myers for Sheriff 2022
1,"""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."",""Stop The Deceptive Corp Prop: Would Turn Every Screen Into A Gambling Device. Learn More.","Coalition for Safe, Responsible Gaming, Sponsored by California Indian Tribes"
2,"""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."",""Stop The Deceptive Corp Prop: Would Turn Every Screen Into A Gambling Device. Learn More.","Coalition for Safe, Responsible Gaming, Sponsored by California Indian Tribes"
3,"""Stand with Captain Sam Brown. Let's restore accountability and leadership to Washington."",""Captain Sam Brown fought for our Country. Now, he is ready to serve the people of Nevada.",SAM BROWN FOR NEVADA
4,"""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."",""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA.","Coalition for Safe, Responsible Gaming, Sponsored by California Indian Tribes"


In [40]:
# check the dataset 
if df_text['text'].isnull().sum():
    print (df_text['text'].isnull().sum())
    df_text = df_text[df_text['text'].notnull()]
    df_text.reset_index(drop=True,inplace=True)

2


In [41]:
df_text = clean_text(df_text)

In [42]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             289 non-null    object
 1   advertiser_name  268 non-null    object
dtypes: object(2)
memory usage: 4.6+ KB


In [10]:
from ipynb.fs.full.helper import get_word_lable

In [ ]:
df_text = get_word_lable(df_text,'lexicon.json')
df_text.head()

In [12]:
df_text.to_csv('ads_ca.csv')

In [13]:
text_topic = count_topic(df_text)
text_topic

,single_topic,counts
0,no topic,139
1,economic,57
2,environment,36
3,donation,32
4,health,26
5,safety,19
6,social programs,18
7,social and cultural,17
8,education,15
9,covid,12


In [14]:
text_word = count_word(df_text)
text_word

,single_word,counts
0,no words,139
1,health,20
2,donate,19
3,tax,12
4,climate,12
...,...,...
167,greenhouse gas,1
168,greenhouse,1
169,genocide,1
170,gas emissions,1


# GOOGLE

In [15]:
if google_df['text'].isnull().sum():
    print (google_df['text'].isnull().sum())
    google_df = google_df[google_df['text'].notnull()]
    google_df.reset_index(drop=True,inplace=True)

In [16]:
google_df = clean_text(google_df)
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       71 non-null     object
 1   ad_url      71 non-null     object
 2   report_url  71 non-null     object
 3   text        71 non-null     object
dtypes: object(4)
memory usage: 2.3+ KB


In [18]:
google_df = get_word_lable(google_df,'lexicon.json')
google_df.head()

,ad_id,ad_url,report_url,text,words,m_label
0,CR221162606398799872,https://transparencyreport.google.com/political-ads/library/advertiser/AR206807382586556416/creative/CR221162606398799872,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR206807382586556416&creative_id=CR221162606398799872&hl=en,ad paid for by dave myers for san diego county sheriff 2022 fppc id 1436219 don t settle for the status quo elect dave myers for sheriff come june 7th 2022,no words,no topic
1,CR436752647349862400,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR436752647349862400,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR436752647349862400&hl=en,a broad coalition of ca indian tribes opposes this online gambling prop learn more stop the deceptive corp prop would turn every screen into a gambling device learn more,no words,no topic
2,CR149929646081703936,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR149929646081703936,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR149929646081703936&hl=en,take action corp prop would legalize an expansion of online sports gambling in ca stop the deceptive corp prop would turn every screen into a gambling device learn more,no words,no topic
3,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada,no words,no topic
4,CR370192611450814464,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR370192611450814464,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR370192611450814464&hl=en,a broad coalition of ca indian tribes opposes this online gambling prop learn more take action corp prop would legalize an expansion of online sports gambling in ca,no words,no topic


In [ ]:
google_topic = count_topic(google_df)
google_topic

In [ ]:
google_word = count_word(google_df)
google_word

# FACEBOOK

In [ ]:
if facebook_df['text'].isnull().sum():
    print (facebook_df['text'].isnull().sum())
    facebook_df = facebook_df[facebook_df['text'].notnull()]
    facebook_df.reset_index(drop=True,inplace=True)

In [ ]:
facebook_df = clean_text(facebook_df)
facebook_df.info

In [ ]:
facebook_df = get_word_lable(facebook_df,'lexicon.json')
facebook_df.head()

In [ ]:
facebook_topic = count_topic(facebook_df)
facebook_topic

In [ ]:
facebook_word = count_word(facebook_df)
facebook_word